In [1]:
#Importando Bibliotecas
import pandas as pd
import numpy as np
import os
from pathlib import Path

In [2]:
#Selecionando as Variáveis
tabua_mortalidade = 'AMERICAN EXPERIENCE'
tabua_mortalidade_inv = 'RRB-44'
tabua_entrada_inv = 'ALVARO VINDAS'
tabua_morbidez = 'Atuários Ingleses'
tx_juros = 0.06
cres_sal = 0.01
cres_benef = 0.01

In [3]:
#Importando as tábuas com os qx
qx_morta = pd.read_csv(str(Path(os.getcwd())) + '\\Tábuas\\tb_mortalidade.csv', encoding='UTF-8', sep=';', usecols=[tabua_mortalidade])
qx_morta_inv = pd.read_csv(str(Path(os.getcwd())) + '\\Tábuas\\tb_mortalidade_inv.csv', encoding='UTF-8', sep=';', usecols=[tabua_mortalidade_inv])
qx_entrada_inv = pd.read_csv(str(Path(os.getcwd())) + '\\Tábuas\\tb_entrada_inv.csv', encoding='UTF-8', sep=';', usecols=[tabua_entrada_inv])
qx_morbidez = pd.read_csv(str(Path(os.getcwd())) + '\\Tábuas\\tb_morbidez.csv', encoding='UTF-8', sep=';', usecols=[tabua_morbidez])

In [4]:
#Calculando os px
px_morta = (1 - qx_morta)
px_morta_inv = (1 - qx_morta_inv)
px_entrada_inv = (1 - qx_entrada_inv)
px_morbidez = (1 - qx_morbidez)

In [5]:
#Calculando os dx e lx
dx_morta = pd.DataFrame(columns=px_morta.columns)
dx_morta_inv = pd.DataFrame(columns=px_morta_inv.columns)
dx_entrada_inv = pd.DataFrame(columns=px_entrada_inv.columns)
dx_morbidez = pd.DataFrame(columns=px_morbidez.columns)

lx_morta = pd.DataFrame(columns=px_morta.columns)
lx_morta_inv = pd.DataFrame(columns=px_morta_inv.columns)
lx_entrada_inv = pd.DataFrame(columns=px_entrada_inv.columns)
lx_morbidez = pd.DataFrame(columns=px_morbidez.columns)

for c in range(127):
    if c >= 1:
        lx_morta.loc[c] = lx_morta.loc[c-1] - dx_morta.loc[c-1]
        lx_morta_inv.loc[c] = lx_morta_inv.loc[c-1] - dx_morta_inv.loc[c-1]
        lx_entrada_inv.loc[c] = lx_entrada_inv.loc[c-1] - dx_entrada_inv.loc[c-1]
        lx_morbidez.loc[c] = lx_morbidez.loc[c-1] - dx_morbidez.loc[c-1]

        dx_morta.loc[c] = lx_morta.loc[c] * qx_morta.loc[c]
        dx_morta_inv.loc[c] = lx_morta_inv.loc[c] * qx_morta_inv.loc[c]
        dx_entrada_inv.loc[c] = lx_entrada_inv.loc[c] * qx_entrada_inv.loc[c]
        dx_morbidez.loc[c] = lx_morbidez.loc[c] * qx_morbidez.loc[c]

    else:
        lx_morta.loc[c] = 100000
        lx_morta_inv.loc[c] = 100000
        lx_entrada_inv.loc[c] = 100000
        lx_morbidez.loc[c] = 100000

        dx_morta.loc[c] = lx_morta.loc[c] * qx_morta.loc[c]
        dx_morta_inv.loc[c] = lx_morta_inv.loc[c] * qx_morta_inv.loc[c]
        dx_entrada_inv.loc[c] = lx_entrada_inv.loc[c] * qx_entrada_inv.loc[c]
        dx_morbidez.loc[c] = lx_morbidez.loc[c] * qx_morbidez.loc[c]


In [6]:
#Calculando as Comutações Dx
Dx_morta = pd.DataFrame(columns=px_morta.columns)

for c in range(127):
    Dx_morta.loc[c] = lx_morta.loc[c]/(1+tx_juros)**c

Dx_morta

,AMERICAN EXPERIENCE
0,100000.000000
1,79745.283019
2,70454.957547
3,64107.364674
4,59032.601495
...,...
122,0.000000
123,0.000000
124,0.000000
125,0.000000


In [7]:
#Calculando a Comutação do Nx
Nx_morta = pd.DataFrame(columns=px_morta.columns)

for c in range(127):
    Nx_morta.loc[c] = Dx_morta.loc[c:].sum()#.map('{:2f}'.format)

Nx_morta

,AMERICAN EXPERIENCE
0,1.206043e+06
1,1.106043e+06
2,1.026298e+06
3,9.558431e+05
4,8.917357e+05
...,...
122,0.000000e+00
123,0.000000e+00
124,0.000000e+00
125,0.000000e+00


In [8]:
#Calculando as ANUIDADES
#Selecionando as Variáveis
x = 30 #Idade
n = 20 #Temporariedade
k = 20 #Diferimento

In [9]:
#Anuidade Antecipada e Vitalícia
#äx = Nx/Dx
ax_ant_vit = Nx_morta.loc[x]/Dx_morta.loc[x]
ax_ant_vit

AMERICAN EXPERIENCE    14.319519
Name: 30, dtype: float64

In [10]:
#Anuidade Postecipada e Vitalícia
#ax = Nx/Dx
ax_pos_vit = Nx_morta.loc[x+1]/Dx_morta.loc[x]
ax_pos_vit

AMERICAN EXPERIENCE    13.319519
dtype: float64

In [11]:
#Anuidade Antecipada e Temporária
#äx/n = Nx - Nx+n/Dx
ax_ant_temp = (Nx_morta.loc[x] - Nx_morta.loc[x+n])/Dx_morta.loc[x]
ax_ant_temp

AMERICAN EXPERIENCE    11.354284
dtype: float64

In [12]:
#Anuidade Postecipada e Temporária
#ax/n = Nx - Nx+n/Dx
ax_pos_temp = (Nx_morta.loc[x+1] - Nx_morta.loc[x+n+1])/Dx_morta.loc[x]
ax_pos_temp

AMERICAN EXPERIENCE    10.609021
dtype: float64

In [13]:
#Anuidade Antecipada, Diferida e Vitalícia
#k/äx = Nx+k/Dx
ax_ant_dif = Nx_morta.loc[x+k]/Dx_morta.loc[x]
ax_ant_dif

AMERICAN EXPERIENCE    2.965235
dtype: float64

In [14]:
#Anuidade Postecipada, Diferida e Vitalícia
#k/ax = Nx+k+1/Dx
ax_pos_dif = Nx_morta.loc[x+k+1]/Dx_morta.loc[x]
ax_pos_dif

AMERICAN EXPERIENCE    2.710498
dtype: float64

In [15]:
#Anuidade Antecipada, Diferida e Temporária
#k/äx/n = Nx+k - Nx+k+n/Dx
ax_ant_dif_temp = (Nx_morta.loc[x+k] - Nx_morta.loc[x+k+n])/Dx_morta.loc[x]
ax_ant_dif_temp

AMERICAN EXPERIENCE    2.67201
dtype: float64

In [16]:
#Anuidade Postecipada, Diferida e Temporária
#k/äx/n = Nx+k - Nx+k+n/Dx
ax_pos_dif_temp = (Nx_morta.loc[x+k+1] - Nx_morta.loc[x+k+n+1])/Dx_morta.loc[x]
ax_pos_dif_temp

AMERICAN EXPERIENCE    2.46116
dtype: float64